In [1]:
import pandas as pd
import sqlite3

conn = sqlite3.connect(r"C:\Users\VTA-HAN\Desktop\VTA\VTA_RM.db")
c = conn.cursor()

In [2]:
from sqlalchemy.orm import Session
from sqlalchemy import create_engine
from sqlalchemy.engine import URL

url_object = URL.create(
    "postgresql",
    username="vta",
    password="vta",
    host="localhost",
    port="5432",
    database="vtadb")
engine = create_engine(url_object)

session = Session(engine)

In [3]:
query = """
SELECT *
    FROM log_table
"""

pd.read_sql_query(query, conn).to_sql('log_table', engine, if_exists='replace')

450

In [5]:
query = """
SELECT *
	FROM exchange_rate
"""
pd.read_sql_query(query, conn).to_csv('exrate.csv', index=False)

In [ ]:
query = """
SELECT from_airport || to_airport AS sector,
		STRFTIME('%Y-%m-%d', depart_date) AS departure_date,
		STRFTIME('%Y-%m-%d', reservation_book_date) AS book_date,
		SUM(CASE 
			WHEN res_seg_status_descr = 'CANCELED' OR ptc = 'INF' THEN 0
			ELSE 1
		END) AS counting
	FROM reservation_segment_detail
	WHERE from_airport IS NOT NULL
	GROUP BY from_airport, to_airport, STRFTIME('%Y-%m-%d', depart_date), STRFTIME('%Y-%m-%d', reservation_book_date)

"""

df = pd.read_sql_query(query, conn)
df.to_csv('res_ve_tet.csv', index=False)

In [ ]:
query = """
SELECT dep_station || arr_station AS sector,
		STRFTIME('%Y-%m-%d', Flight_Origin_Date_LT) AS departure_date,
		SUM(SUBSTRING(config, -3)*1) AS capacity,
		COUNT(*) AS frequency 
	FROM flown_aircraft_leg
	WHERE Flight_Origin_Date_LT >= '2023-01-06' AND Flight_Origin_Date_LT <= '2023-02-06'
		AND flight_status IN ('OPEN', 'CLOSED')
	GROUP BY dep_station, arr_station, STRFTIME('%Y-%m-%d', Flight_Origin_Date_LT)
"""

df = pd.read_sql_query(query, conn)
df.to_csv('leg_tet.csv', index=False)

In [4]:
query = """
WITH 
range_date AS (
SELECT STRFTIME('%Y-%m-%d', Date) AS range_date 
	FROM dim_calendar
	WHERE (lunar_date BETWEEN '2022_12_15' AND '2023_01_15')
		OR (lunar_date BETWEEN '2021_12_15' AND '2022_01_15')),
		
regular_flight AS (
SELECT flight_num
	FROM flight_type
	WHERE flight_type = 'REGULAR')

SELECT 	from_airport || to_airport AS sector,
		STRFTIME('%Y-%m-%d', depart_date) AS departure_date,
		STRFTIME('%Y-%m-%d', reservation_book_date) AS book_date,
		SUM(CASE 
			WHEN res_seg_status_descr = 'CANCELED' OR ptc = 'INF' THEN 0
			ELSE 1
		END) AS counting
	FROM reservation_segment_detail r
	WHERE from_airport IS NOT NULL
		AND (STRFTIME('%Y-%m-%d', depart_date) IN (SELECT range_date FROM range_date))
		AND (r.operating_flight_num IN (SELECT flight_num FROM regular_flight))
	GROUP BY from_airport, to_airport, STRFTIME('%Y-%m-%d', depart_date), STRFTIME('%Y-%m-%d', reservation_book_date)

"""

df = pd.read_sql_query(query, conn)
df.to_csv('res_ve_tet_2.csv', index=False)

In [11]:
pd.DataFrame({'refresh_datetime':[pd.Timestamp.now()]})

,refresh_datetime
0,2023-01-30 10:55:51.117916
